In [ ]:
from tqdm import tqdm
import argparse
from utils import *
from freebase import *
from propagation import *
import random



parser = argparse.ArgumentParser()
parser.add_argument("--dataset", type=str,
                    default="cwq", help="choose the dataset from {webqsp, cwq}.")
parser.add_argument("--max_length", type=int,
                    default=1024, help="the max length of LLMs output.")
parser.add_argument("--temperature", type=float,
                    default=0., help="the temperature")
parser.add_argument("--llm", type=str,
                    default="llama-3", help="choose base LLM model from {llama, gpt-3.5-turbo, gpt-4}.")
parser.add_argument("--openai_api_key", type=str,
                    default="", help="if the LLM is gpt-3.5-turbo or gpt-4, you need add your own openai api key.")
parser.add_argument('--verbose', action='store_true', help="print LLM input and output.")
args = parser.parse_args(["--verbose"])


datas, question_string = prepare_dataset(args.dataset)

In [ ]:
data = datas[3]
question = data[question_string]
topics = data['topic_entity']
paths = {topics[topic]: {} for topic in topics}
print(question)

In [ ]:
# for topic in topics:
for topic in topics:
    topic_name = topics[topic]
    # 1-hop propagation
    relations = get_relations(question, topic, topic_name, args, 3)
    entities_id, entities_name = get_entities(topic, relations)
    [paths[topic_name].update({r: {"entities_id": entities_id[i], "entities_name": entities_name[i]}}) for i, r in enumerate(relations)]
    facts = propagate(question, topic_name, relations, paths[topic_name], 1, args)
    [paths[topic_name][r].update({"fact": facts[i]}) for i, r in enumerate(relations)]
    # 2-hop propagation
    relations = get_relations_distant(question, topic, topic_name, relations, paths[topic_name], args, 3)
    entities_id, entities_name = get_entities_distant(paths[topic_name], relations)
    [paths[topic_name].update({r: {"entities_id": entities_id[i], "entities_name": entities_name[i]}}) for i, r in enumerate(relations)]
    facts = propagate(question, topic_name, relations, paths[topic_name], 2, args)
    [paths[topic_name][r].update({"fact": facts[i], "entities_id": sum(entities_id[i], []), "entities_name": sum(entities_name[i], [])}) for i, r in enumerate(relations)]
    # # 3-hop propagation
    relations = get_relations_distant(question, topic, topic_name, relations, paths[topic_name], args, 3)
    entities_id, entities_name = get_entities_distant(paths[topic_name], relations)
    [paths[topic_name].update({r: {"entities_id": entities_id[i], "entities_name": entities_name[i]}}) for i, r in enumerate(relations)]
    facts = propagate(question, topic_name, relations, paths[topic_name], 3, args)
    [paths[topic_name][r].update({"fact": facts[i]}) for i, r in enumerate(relations)]
    # # clean paths
    [paths[topic_name].update({r: paths[topic_name][r]['fact']}) for r in paths[topic_name]]


===================input======================
Given the question, we have some new detailed facts about 1 previously summarized facts.

question: what countries share borders with france and is the location contains and airport that server Nijmegen

Based on the question and previously summarized facts, please summarize each following new detailed fact to better answer question and just return all summarized 1 facts as following order in a numbered list. 

1. 
Previous summarized fact: France has many administrative divisions, including departments and regions, but none of them are directly related to Nijmegen.
New detailed fact: The topic Paris has relation location.location.contains with following entities: [16th arrondissement; 2nd arrondissement; 3rd arrondissement; 12th arrondissement; 6th arrondissement of Paris; Saint-Nicolas-du-Chardonnet; Pantheon-Sorbonne University; Tour Michelet; Gare de Javel; 19th arrondissement; Le Trabendo; Tour Triangle; University of Paris III: Sorbo

In [ ]:
facts = construct_facts(paths, topics)
prompt = question_prompt.format(facts, question)
response = run_llm(prompt, args.temperature, args.max_length, args.openai_api_key, args.llm, args.verbose)
output = {"question": question, "result": response, "paths": paths}

In [ ]:
relations